### Importing necessary libraries

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window

### Loading data

In [0]:
sc = spark.sparkContext
path = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
steam_df = spark.read.json(path)

In [0]:
num_rows = steam_df.count()
print(f"The DataFrame contains {num_rows} rows.")

The DataFrame contains 55691 rows.


First of all, we need to check the structure of our data. That's why we print the Schema.

At the root level, the data is separated into two columns, id and data. We are only interested in the data column, since there's an id nested inside the data column.

Moreover, it seems that each record represents a video game on the platform.

In [0]:
steam_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
steam_df.show()

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
|{1000080, [Multi-...|1000080|
|{1000100, [Single...|1000100|
|{1000110, [Multi-...|1000110|
|{1000130, [Single...|1000130|
|{1000280, [Single...|1000280|
|{1000310, [Multi-...|1000310|
|{1000360, [Multi-...|1000360|
|{1000370, [Single...|1000370|
|{1000380, [Single...|1000380|
|{1000410, [Single...|1000410|
|{1000470, [Single...|1000470|
|{1000480, [Single...|1000480|
|{1000500, [Multi-...|1000500|
|{1000510, [], 0, ...|1000510|
|{1000540, [Multi-...|1000540|
+--------------------+-------+
only showing top 20 rows



In [0]:
steam_df = steam_df.select(steam_df.data)

Further analysis, seems to suggest that a lot of tags are nested inside the data structure. Tags are an important source of information as it helps users to find similar interests as well as recommender systems.

Tags require a separate in depth analysis in order to extract valuable insights from them.

We'll focus our analysis on the rest of the features.

In [0]:
steam_df = steam_df.select(steam_df.data.dropFields('tags').alias('data'))

In [0]:
steam_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
# How to rename columns
#steam_df = steam_df.withColumnRenamed("update_fields(data, dropfield())", "data")

### Macro level analysis:

#### Which publisher has released the most games on Steam?

We will start by checking the publisher with the highest number of released games on Steam. We will show the top 10 publishers as well.
The top publisher is Big Fish Games. Interestingly, in the Top 10, we can see an empty field, which means there are some games without publishers.

In [0]:
df = steam_df.select(steam_df.data.getField("publisher").alias("publisher"))
publisher_counts = df.groupBy("publisher").count()
publisher_with_most_games = publisher_counts.sort(F.desc("count")).first()

print(f"The publisher with the most games is: {publisher_with_most_games['publisher']}")

top_10_publishers = publisher_counts.sort(F.desc("count")).limit(10)
print("Top 10 publishers on steam based on number of released games on Steam are:")
top_10_publishers = top_10_publishers.withColumn('publisher', 
                   F.when(F.length(F.trim(F.col('publisher'))) == 0, 'no publisher')
                    .otherwise(F.col('publisher')))
display(top_10_publishers)
#top_10_publishers.show()

The publisher with the most games is: Big Fish Games
Top 10 publishers on steam based on number of released games on Steam are:


publisher,count
Big Fish Games,422
8floor,202
SEGA,165
Strategy First,151
Square Enix,141
Choice of Games,140
Sekai Project,132
HH-Games,132
no publisher,132
Ubisoft,127


Databricks visualization. Run in Databricks to view.

#### What are the best rated games?

To answer this question, we first need to define what "best rated" means. In order to decide, we will focus on two particular Fields:

positive and negative. Steam allows users to give a thumbs up (positive) or a thumbs down (negative) to a game they own. For the best rated games, we will focus on the positive Field.

In [0]:
games = steam_df.select(steam_df["data.name"].alias("video_game"),steam_df["data.positive"].alias("positive_ratings"))
top_rated_games = games.sort(F.desc("positive_ratings")).limit(10)
display(top_rated_games)

video_game,positive_ratings
Counter-Strike: Global Offensive,5943345
Dota 2,1534895
Grand Theft Auto V,1229265
PUBG: BATTLEGROUNDS,1185361
Terraria,1014711
Tom Clancy's Rainbow Six Siege,942910
Garry's Mod,861240
Team Fortress 2,846407
Rust,732513
Left 4 Dead 2,643836


Databricks visualization. Run in Databricks to view.

We can see that a lot of Valve's own games are present in the highest rated games such as `Counter-Strike`, `Dota 2`, `Team Fortress 2` and `Left 4 Dead 2`

`Counter-Strike` is the highest rated game. It is a very popular game whithin the gaming community.

#### Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

To answer this question, we first need to define what "best rated" means. In order to decide, we will focus on two particular Fields:

positive and negative. Steam allows users to give a thumbs up (positive) or a thumbs down (negative) to a game they own. For the best rated games, we will focus on the positive Field.

In [0]:
release_year = steam_df.select(F.year(F.to_date(steam_df["data.release_date"], format="yyyy/MM/d")).alias("release_year"))
games_per_year = release_year.groupBy("release_year").count()
games_per_year_sorted = games_per_year.sort(F.desc("release_year")).limit(10)
display(games_per_year_sorted)

release_year,count
2022,7451
2021,8805
2020,8287
2019,6949
2018,7663
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

There are definitely years with more releases than others. This might be due to the video game development cycle. Video games take sometimes many years to be developed. Sometimes video games might be delayed even due to technical issues or unsatisfying performance.

Interestingly enough, the covid period (2020-2021) have had more releases than all the other years in the last 20 years.

#### How are the prices distributed? Are there many games with a discount?

In [0]:
prices = steam_df.select(steam_df["data.price"].alias("price"),steam_df["data.initialprice"].alias("initialprice"))
prices.describe().show()

+-------+-----------------+-----------------+
|summary|            price|     initialprice|
+-------+-----------------+-----------------+
|  count|            55691|            55691|
|   mean|773.2849832109317|797.5663033524268|
| stddev| 1093.13458272345|1104.762477841338|
|    min|                0|                0|
|    max|             9999|             9999|
+-------+-----------------+-----------------+



To answer this question, we first need to visualize the distribution of the prices. Then we have to compare the initial price with the actual price. If the price is less than the initial price, then said game is discounted.

One thing we noticed while analysing the prices is that there seems to be a parsing error. The values are unreasonable for video games. We assume a point or a comma was lost somewhere and we divide by 100 after casting the string values to float. We obtain much more reasonable prices.

There also seems to be an outlier, as even after dividing by a 100 and obtaining reasonable prices, we still find a 999 value. We will consider it as an outlier and filter it.

By further analysing the prices we noticed that most video games are priced in the 0-20 (we assume USD) price range. For better visualization, we limit our data to a 50 USD threshold.

Last but not least. To check if the games are currently discounted or not, we create a new column where we compare the price to the initial price.

In [0]:
prices = steam_df.select(steam_df["data.price"].alias("price"),steam_df["data.initialprice"].alias("initialprice"))
prices = prices.withColumn("price", prices["price"].cast(T.FloatType())/100.0)
prices = prices.withColumn("initialprice", prices["initialprice"].cast(T.FloatType())/100.0)
prices = prices.filter(prices["price"]<50)
display(prices)


price,initialprice
9.99,9.99
9.99,9.99
5.99,19.99
19.99,19.99
1.99,1.99
7.99,19.99
12.99,12.99
0.0,0.0
2.99,2.99
13.99,13.99


Databricks visualization. Run in Databricks to view.

In [0]:
# Create a new field for whether the game is discounted
prices = prices.withColumn('discounted', F.when(F.col('price') < F.col('initialprice'), "discount").otherwise("no discount"))

# Count the number of games with and without discounts
discount_counts = prices.groupBy('discounted').count()

display(discount_counts)

discounted,count
discount,2516
no discount,52847


Databricks visualization. Run in Databricks to view.

#### What are the most represented languages?

To answer this question, we need to count the number of games available in each language. A lot of games come in different languages.

English, German and French are the top 3 supported languages.

In [0]:
df_languages = steam_df.select(F.explode(F.split(steam_df['data.languages'], ', ')).alias('language'))

# Count the number of games per language
language_counts = df_languages.groupBy('language').count()

# Show the top 10 most represented languages
display(language_counts.sort('count', ascending=False).limit(10))

language,count
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6599


Databricks visualization. Run in Databricks to view.

#### Are there many games prohibited for children under 16/18?

To answer this question, we need to count the number of games available in each language. A lot of games come in different languages

We first check the values in the required_age field. The highest count is 0 which we assume means no restriction. We have some unusual values such as `180`, `35`, `21+`and `7+`these coupled with `16`, `17` and `18` can be considered as games restricted to children.

There doesn't seem to be many prohibited games for children.

In [0]:
age_df = steam_df.select(steam_df['data.required_age'].alias("required_age"))
# Define the values that indicate a game is prohibited for children
prohibited_values = ['180', '35', '21+', '7+', '16', '17', '18']
age_df = age_df.withColumn('prohibited', F.when(F.col('required_age').isin(prohibited_values), "prohibited").otherwise("not prohibited"))
prohibited_count = age_df.groupBy('prohibited').count()
display(prohibited_count.sort('count', ascending=False))

prohibited,count
not prohibited,55385
prohibited,306


Databricks visualization. Run in Databricks to view.

### Genres analysis:

#### What are the most represented genres?

To answer this question, we would need to count the number of games in each genre, similarly to the languages.

Indie dominates the most represented genres

In [0]:
df_genres = steam_df.select(F.explode(F.split(steam_df['data.genre'], ', ')).alias('genre'))

# Count the number of games per language
genre_counts = df_genres.groupBy('genre').count()

# Show the top 10 most represented languages
display(genre_counts.sort('count', ascending=False).limit(10))

genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

#### Are there any genres that have a better positive/negative review ratio?

To answer this question, we need to calculate the positive/negative review ratio for each genre.

Surprisingly, it's not the most represented genres that have the better positive/negative review ratio. Only Indie is present in the top 10 from the best represented genres.

In [0]:
df_genres = steam_df.select(F.explode(F.split(steam_df['data.genre'], ', ')).alias('genre'),
                            steam_df["data.positive"].alias("positive"),
                            steam_df["data.negative"].alias("negative"))

# Calculate the total positive and negative reviews for each genre
genre_review_totals = df_genres.groupBy('genre').agg(F.sum('positive').alias('total_positive'), F.sum('negative').alias('total_negative'))

# Calculate the review ratio for each genre
genre_review_ratios = genre_review_totals.withColumn('review_ratio', F.col('total_positive') / F.col('total_negative'))

genre_review_ratios = genre_review_ratios.withColumn('genre', 
                   F.when(F.length(F.trim(F.col('genre'))) == 0, 'No genre')
                    .otherwise(F.col('genre')))

# Show the genres with the best review ratio
display(genre_review_ratios.sort('review_ratio', ascending=False).limit(10))

genre,total_positive,total_negative,review_ratio
Photo Editing,577751,13745,42.03353946889778
Animation & Modeling,690765,26392,26.17327220369809
Design & Illustration,674057,27007,24.958603325063873
Utilities,739335,43503,16.99503482518447
Game Development,27461,3274,8.38759926695174
Indie,32531023,4241234,7.67017877344188
Audio Production,69118,9428,7.331141281289775
No genre,189649,27744,6.835676182237601
Video Production,111514,16362,6.815425987043149
Casual,10034967,1537296,6.527673915758578


Databricks visualization. Run in Databricks to view.

#### Do some publishers have favorite genres?

To answer this question, we need to find the most common genre for each publisher. This involves grouping the data by publisher and genre, counting the number of games for each combination, and then finding the genre with the most games for each publisher.

Since there are many publishers, we will focus on the biggest publishers. Most of them prefer Casual games followed by Action games and lastly RPG and Strategy.

In [0]:
df_genres = steam_df.select(F.explode(F.split(steam_df['data.genre'], ', ')).alias('genre'),
                            steam_df["data.publisher"].alias("publisher"))

# Convert the DataFrame of top 10 publishers to a list
top_10_publishers_list = top_10_publishers.rdd.flatMap(lambda x: x).collect()

# Count the number of games for each publisher-genre combination
publisher_genre_counts = df_genres.groupBy('publisher', 'genre').count()


# Create a window partitioned by publisher and ordered by count in descending order
window = Window.partitionBy('publisher').orderBy(F.col('count').desc())

# Add a row number within each window partition
publisher_genre_counts = publisher_genre_counts.withColumn('rank', F.row_number().over(window))

# Get the favorite genre for each publisher (the genre with the most games)
favorite_genres = publisher_genre_counts.filter(F.col('rank') == 1).select('publisher', 'genre')

# Filter the favorite_genres DataFrame for the top 10 publishers
favorite_genres_top_10 = favorite_genres.filter(F.col('publisher').isin(top_10_publishers_list))

favorite_genres_top_10.show()

+---------------+--------+
|      publisher|   genre|
+---------------+--------+
|         8floor|  Casual|
| Big Fish Games|  Casual|
|Choice of Games|     RPG|
|       HH-Games|  Casual|
|           SEGA|  Action|
|  Sekai Project|  Casual|
|    Square Enix|  Action|
| Strategy First|Strategy|
|        Ubisoft|  Action|
+---------------+--------+



#### What are the most lucrative genres?

To answer this question, we can make the assumption that the most lucrative genres are the ones with the highest total revenue. However, we don't have direct information about revenue, but we have the price of each game and an owners range. We have the owners Field which is a String like this: "nb1 .. nb2". We need to get the mean of these two numbers and multiply it by the price. We'll call it estimated total gross. Then we'll group by genre.

In [0]:
df_genres = steam_df.select(steam_df['data.genre'].alias('genre'),
                             steam_df['data.owners'].alias('owners'),
                              steam_df['data.price'].alias('price'))

# Split the owners field into two columns
df_genres = df_genres.withColumn("owners_start", F.expr("float(regexp_replace(split(owners, ' .. ')[0], ',', ''))"))
df_genres = df_genres.withColumn("owners_end", F.expr("float(regexp_replace(split(owners, ' .. ')[1], ',', ''))"))
df_genres = df_genres.withColumn("price", df_genres["price"].cast(T.FloatType())/100.0)
# Compute the average of the owners_start and owners_end, and multiply by the price
df_genres = df_genres.withColumn("estimated_gross", (df_genres["owners_start"] + df_genres["owners_end"]) / 2 * df_genres["price"])
df_genres = df_genres.withColumn('genre', F.explode(F.split(F.col('genre'), ', ')))
# Calculate the total estimated gross for each genre
genre_gross = df_genres.groupBy('genre').sum('estimated_gross')

# Show the genres with the highest total estimated gross
display(genre_gross.sort('sum(estimated_gross)', ascending=False).limit(10))

genre,sum(estimated_gross)
Action,5.87564541E10
Adventure,3.724573845E10
Indie,3.23465771E10
RPG,2.71731431E10
Strategy,2.015004105E10
Simulation,1.876974955E10
Casual,8.08095665E9
Massively Multiplayer,5.93015775E9
Early Access,5.45866145E9
Sports,3.14989735E9


Databricks visualization. Run in Databricks to view.

### Platform analysis

#### Are most games available on Windows/Mac/Linux instead?

To answer this question, we need to count the number of games available on each platform. If the platform information for each game is stored in separate fields (e.g., a windows field, a mac field, and a linux field), and these fields contain boolean values indicating whether the game is available on that platform, we can simply sum these fields to get the total number of games available on each platform.

Without no surprise, windows is by far the leading platform in terms of games available.

In [0]:
# Count the number of games available on each platform
platform_counts = steam_df.agg(F.sum(steam_df['data.platforms.windows'].cast("int")).alias('windows_count'),
                         F.sum(steam_df['data.platforms.mac'].cast("int")).alias('mac_count'),
                         F.sum(steam_df['data.platforms.linux'].cast("int")).alias('linux_count'))

display(platform_counts)

windows_count,mac_count,linux_count
55676,12770,8458


Databricks visualization. Run in Databricks to view.

#### Do certain genres tend to be preferentially available on certain platforms?

To answer this question, we need to count the number of games in each genre that are available on each platform. This involves grouping the data by genre and platform, and counting the number of games for each combination. We lastly compare the values for each platform, and choose the platform with the highest count.

Windows is again the preferred platform for all genres.

In [0]:
# Split the genres field into separate rows for each genre
df_genres = steam_df.select(F.explode(F.split(steam_df['data.genre'], ', ')).alias('genre'),
                             steam_df['data.platforms.windows'],
                              steam_df['data.platforms.mac'],
                               steam_df['data.platforms.linux'])

# Count the number of games available on each platform for each genre
genre_platform_counts = df_genres.groupBy('genre').agg(F.sum(df_genres['windows'].cast("int")).alias('windows_count'),
                         F.sum(df_genres['mac'].cast("int")).alias('mac_count'),
                         F.sum(df_genres['linux'].cast("int")).alias('linux_count'))

genre_platform_counts = genre_platform_counts.withColumn('platform_choice', 
                                                         F.when((F.col('windows_count') >= F.col('mac_count')) & 
                                                              (F.col('windows_count') >= F.col('linux_count')), 'windows')
                                                         .when((F.col('mac_count') >= F.col('windows_count')) & 
                                                               (F.col('mac_count') >= F.col('linux_count')), 'mac')
                                                         .otherwise('linux'))


display(genre_platform_counts)

genre,windows_count,mac_count,linux_count,platform_choice
Education,317,56,19,windows
Massively Multiplayer,1459,270,164,windows
Sexual Content,54,13,7,windows
Adventure,21427,5039,3302,windows
Sports,2665,506,287,windows
Accounting,16,4,0,windows
Audio Production,193,41,7,windows
Video Production,247,29,6,windows
Animation & Modeling,322,74,38,windows
Racing,2154,424,304,windows
